<a href="https://colab.research.google.com/github/senushidinara/LearnScape-AI/blob/cosmos-world/Wolfram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from IPython.display import Image, display
import json

# ---------------------------------------------
# CONFIG — paste your Wolfram AppID
# ---------------------------------------------
API_KEY = "H34WX632E2"

# ---------------------------------------------
# MAIN FUNCTION — full processing pipeline
# ---------------------------------------------
def wolfram_all(query):
    print("🔍 Querying Wolfram Alpha...")

    # Build URL
    url = (
        f"https://api.wolframalpha.com/v2/query"
        f"?input={query}"
        f"&appid={API_KEY}"
        f"&output=json"
    )

    # Request JSON
    data = requests.get(url).json()

    # Prepare result container
    output = {
        "query": query,
        "pods": [],
        "images": [],
        "plots": [],
        "steps": [],
        "text_blocks": []
    }

    # Extract pods
    pods = data["queryresult"].get("pods", [])

    for pod in pods:
        title = pod["title"]
        subpod = pod["subpods"][0]

        # Save text content
        plaintext = subpod.get("plaintext", "")
        if plaintext:
            output["text_blocks"].append({"title": title, "text": plaintext})

        # Save images
        if "img" in subpod:
            img_url = subpod["img"]["src"]
            output["images"].append({"title": title, "url": img_url})

            # If it's a plot, store separately
            if "Plot" in title or "plot" in title.lower():
                output["plots"].append(img_url)

        # Step-by-step solutions
        if "Step-by-step solution" in title:
            output["steps"].append(plaintext)

        # Store raw pod
        output["pods"].append({
            "title": title,
            "plaintext": plaintext,
            "image": subpod.get("img", {}).get("src", None)
        })

    # Display images directly in Colab
    print("\n📊 Showing visual outputs:\n")
    for img in output["images"]:
        print("Image:", img["title"])
        display(Image(url=img["url"]))

    print("\n✅ Finished.")
    return output


# ---------------------------------------------
# RUN EVERYTHING WITH ONE COMMAND
# ---------------------------------------------
query = "definite integral of x^2 + 2x from 0 to 5"
result = wolfram_all(query)

# Show the cleaned LearnScape-ready dictionary
print("\n============= LEARNSCAPE OUTPUT =============\n")
print(json.dumps(result, indent=2))

🔍 Querying Wolfram Alpha...

📊 Showing visual outputs:

Image: Definite integral


Image: Visual representation of the integral


Image: Riemann sums


Image: Indefinite integral



✅ Finished.

============= LEARNSCAPE OUTPUT =============

{
  "query": "definite integral of x^2 + 2x from 0 to 5",
  "pods": [
    {
      "title": "Definite integral",
      "plaintext": "integral_0^5 x^2 2 x dx = 625/2 = 312.5",
      "image": "https://public5c.wolframalpha.com/files/GIF_xq5ydkim5v.gif"
    },
    {
      "title": "Visual representation of the integral",
      "plaintext": "",
      "image": "https://public5c.wolframalpha.com/files/GIF_xq661p7oc2.gif"
    },
    {
      "title": "Riemann sums",
      "plaintext": "left sum | (625 (n - 1)^2)/(2 n^2) = 625/2 - 625/n + O((1/n)^2)\n(assuming subintervals of equal length)",
      "image": "https://public5c.wolframalpha.com/files/GIF_xq61xvg6cp.gif"
    },
    {
      "title": "Indefinite integral",
      "plaintext": "integral x^2 2 x dx = x^4/2 + constant",
      "image": "https://public5c.wolframalpha.com/files/GIF_xq5zwgenc2.gif"
    }
  ],
  "images": [
    {
      "title": "Definite integral",
      "url": "https

In [2]:
import requests
import json

# YOUR APP ID
APP_ID = "H34WX632E2"

def wolfram_query(query):
    """Send query to Wolfram Alpha Full Results API"""
    url = "https://api.wolframalpha.com/v2/query"

    params = {
        "input": query,
        "appid": APP_ID,
        "output": "JSON",         # important: return JSON not XML
        "format": "image,plaintext"
    }

    response = requests.get(url, params=params)
    data = response.json()

    return data


def parse_wolfram_result(data):
    """Extract clean results from Wolfram JSON"""

    pods = data.get("queryresult", {}).get("pods", [])
    result = {
        "input_interpretation": None,
        "plaintext_results": [],
        "images": [],
        "3d_data": None
    }

    for pod in pods:
        title = pod.get("title", "").lower()

        # Text results / formulas / steps
        if pod.get("subpods"):
            for sp in pod["subpods"]:
                if "plaintext" in sp and sp["plaintext"]:
                    result["plaintext_results"].append({
                        "title": pod.get("title"),
                        "value": sp["plaintext"]
                    })

                # Image results (plots)
                if "img" in sp and "src" in sp["img"]:
                    result["images"].append(sp["img"]["src"])

        # Try extracting 3D data if available
        if "3d" in title or "plot" in title:
            try:
                # Wolfram sometimes puts JSON 3D data inside "minput" or "moutput"
                result["3d_data"] = pod
            except:
                pass

    return result


def pretty_print(result):
    print("==== TEXT RESULTS ====\n")
    for item in result["plaintext_results"]:
        print(f"{item['title']}: {item['value']}\n")

    print("==== IMAGES ====\n")
    for img in result["images"]:
        print(img)

    print("\n==== 3D DATA (raw) ====\n")
    print(json.dumps(result["3d_data"], indent=2))


# -----------------------------
#        RUN EVERYTHING
# -----------------------------
query = "plot x^2 + 2x from -5 to 5 in 3D"
data = wolfram_query(query)
result = parse_wolfram_result(data)
pretty_print(result)

==== TEXT RESULTS ====

Input interpretation: 3D plot | x^2 + 2 x | x = -5 to 5

==== IMAGES ====

https://public6.wolframalpha.com/files/GIF_xtj90b8pgn.gif
https://public6.wolframalpha.com/files/GIF_xtjnqya4pw.gif

==== 3D DATA (raw) ====

null


In [3]:
# ONE-CELL: Full Wolfram -> LearnScape mini-app for Colab
# Run in Google Colab. Replace APP_ID below if needed.

# -----------------------
# Install & runtime setup
# -----------------------
!pip install -q fastapi "uvicorn[standard]" nest_asyncio pyngrok requests

import nest_asyncio, uvicorn, json, asyncio, hashlib, time
from fastapi import FastAPI, Request, Form
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.staticfiles import StaticFiles
from threading import Thread
import requests
from pyngrok import ngrok
nest_asyncio.apply()

APP_ID = "H34WX632E2"   # ← Your Wolfram AppID (already set). Replace if you want.

# -----------------------
# Helper: Wolfram query
# -----------------------
def wolfram_query_raw(query: str, appid: str=APP_ID, output="JSON"):
    url = "https://api.wolframalpha.com/v2/query"
    params = {
        "input": query,
        "appid": appid,
        "output": output,
        "format": "image,plaintext"
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

# -----------------------
# Core generator function
# -----------------------
# Returns dict with: visualization_data, interactive_elements, learning_insights, threejs_config
_in_memory_cache = {}

def generate_math_visualization(query: str, concept_context: str = ""):
    """
    Query Wolfram, parse, create a LearnScape-friendly payload.
    Uses image fallback when no mesh/points available.
    """
    # cache key
    h = hashlib.sha1((query + "|" + concept_context).encode()).hexdigest()
    if h in _in_memory_cache:
        return _in_memory_cache[h]

    try:
        raw = wolfram_query_raw(query)
    except Exception as e:
        return {"error": f"Wolfram query failed: {repr(e)}"}

    # parse pods
    pods = raw.get("queryresult", {}).get("pods", [])

    plaintexts = []
    images = []
    numeric_tables = []  # if we find tables of numbers (x,y,z) -> convert to points
    threejs_config = {"geometries": [], "materials": [], "objects": []}

    for pod in pods:
        title = pod.get("title", "")
        for sub in pod.get("subpods", []):
            txt = sub.get("plaintext", "")
            if txt:
                plaintexts.append({"title": title, "text": txt})

            img = sub.get("img", {})
            if img and img.get("src"):
                images.append({"title": title, "url": img["src"]})

            # look for structured numeric data in plaintext (simple heuristics)
            if txt and ("\t" in txt or "\n" in txt):
                # try parse tabular numeric lines
                rows = [r.strip() for r in txt.splitlines() if r.strip()]
                parsed = []
                is_numeric_table = True
                for row in rows:
                    parts = row.replace(",", " ").split()
                    try:
                        nums = [float(p) for p in parts]
                        parsed.append(nums)
                    except:
                        is_numeric_table = False
                        break
                if is_numeric_table and parsed:
                    numeric_tables.append({"title": title, "table": parsed})

    # Convert numeric tables to ThreeJS point cloud if possible
    if numeric_tables:
        # take the largest numeric table as geometry
        table = max(numeric_tables, key=lambda t: len(t["table"]))["table"]
        # Normalize into float triplets (if only 2D given, add z=0)
        points = []
        for row in table:
            if len(row) >= 3:
                x,y,z = row[0], row[1], row[2]
            elif len(row) == 2:
                x,y,z = row[0], row[1], 0.0
            else:
                continue
            points.append([x,y,z])
        # Build simple geometry payload
        geometry = {
            "type": "pointcloud",
            "positions": points,
            "metadata": {"source": "wolfram_table"}
        }
        threejs_config["geometries"].append(geometry)
        threejs_config["materials"].append({"type":"points","size":0.05})
        threejs_config["objects"].append({"geometry_index":0,"material_index":0,"type":"Points"})
    else:
        # No numeric table: fallback to image plot if exists
        if images:
            # use the first Plot-like image as a textured plane in Three.js
            plot_url = images[0]["url"]
            # plane geometry payload
            geometry = {"type":"plane","size":[4,3],"metadata":{"source":"wolfram_image"}}
            material = {"type":"texture","map":plot_url}
            obj = {"geometry_index":0,"material_index":0,"type":"Mesh","transform":{}}
            threejs_config["geometries"].append(geometry)
            threejs_config["materials"].append(material)
            threejs_config["objects"].append(obj)
        else:
            # last-resort: instruct front-end to render text
            pass

    # Build learning insights (very lightweight here — extendable)
    learning_insights = {
        "summary": plaintexts[0]["text"] if plaintexts else "",
        "all_texts": plaintexts,
        "image_count": len(images),
        "numeric_table_count": len(numeric_tables)
    }

    result = {
        "query": query,
        "concept_context": concept_context,
        "visualization_data": {"images": images, "plaintexts": plaintexts, "numeric_tables": numeric_tables},
        "interactive_elements": {"sliders": [{"name":"param","min":-10,"max":10,"step":0.1}]},
        "learning_insights": learning_insights,
        "threejs_config": threejs_config,
        "raw_wolfram": raw
    }

    # cache for 10 minutes
    _in_memory_cache[h] = result
    return result

# -----------------------
# FastAPI app + endpoints
# -----------------------
app = FastAPI()

# Serve a minimal front-end (Three.js + simple UI)
INDEX_HTML = """
<!doctype html>
<html>
<head>
  <meta charset="utf-8">
  <title>LearnScape — Wolfram Visualizer</title>
  <style>body{margin:0;font-family:sans-serif} #ui{position:absolute;left:8px;top:8px;background:rgba(255,255,255,0.9);padding:10px;border-radius:6px;z-index:10}</style>
</head>
<body>
<div id="ui">
  <div><b>Wolfram Visualizer (Colab)</b></div>
  <div>
    <input id="query" style="width:320px" value="definite integral of x^2 + 2x from 0 to 5"/>
    <button onclick="visualize()">Visualize</button>
  </div>
  <div id="info"></div>
</div>
<canvas id="c"></canvas>

<!-- Load Three.js from CDN -->
<script src="https://cdn.jsdelivr.net/npm/three@0.146.0/build/three.min.js"></script>
<script>
const canvas = document.getElementById('c');
const renderer = new THREE.WebGLRenderer({canvas});
renderer.setSize(window.innerWidth, window.innerHeight);
const scene = new THREE.Scene();
scene.background = new THREE.Color(0xf0f0f0);
const camera = new THREE.PerspectiveCamera(60, window.innerWidth/window.innerHeight, 0.1, 1000);
camera.position.set(0,0,6);
const light = new THREE.DirectionalLight(0xffffff, 0.9); light.position.set(1,1,1); scene.add(light);

let currentObjects = [];
function clearScene(){
  for(let o of currentObjects){ scene.remove(o); if(o.geometry) o.geometry.dispose(); if(o.material) o.material.dispose(); }
  currentObjects = [];
}

async function visualize(){
  const q = document.getElementById('query').value;
  document.getElementById('info').innerText = "Querying...";
  const res = await fetch('/api/visualize', {
    method:'POST', headers:{'Content-Type':'application/json'}, body: JSON.stringify({query:q,context:"auto"})
  });
  const data = await res.json();
  document.getElementById('info').innerText = "Images: " + (data.visualization_data.images.length) + " | Texts: " + (data.visualization_data.plaintexts.length);
  clearScene();
  const cfg = data.threejs_config;

  if(cfg.geometries && cfg.geometries.length>0){
    for(let i=0;i<cfg.geometries.length;i++){
      const geom = cfg.geometries[i];
      const mat = cfg.materials[i] || {};
      if(geom.type === "pointcloud"){
        const positions = new Float32Array(geom.positions.flat());
        const geometry = new THREE.BufferGeometry();
        geometry.setAttribute('position', new THREE.BufferAttribute(positions, 3));
        const material = new THREE.PointsMaterial({size:0.08, sizeAttenuation:true});
        const points = new THREE.Points(geometry, material);
        scene.add(points);
        currentObjects.push(points);
      } else if(geom.type === "plane"){
        const size = geom.size || [4,3];
        const planeGeo = new THREE.PlaneGeometry(size[0], size[1]);
        // material from texture
        const matSpec = cfg.materials[i] || {};
        if(matSpec.type === "texture" && matSpec.map){
          const tex = new THREE.TextureLoader().load(matSpec.map);
          const m = new THREE.MeshBasicMaterial({map:tex});
          const mesh = new THREE.Mesh(planeGeo, m);
          scene.add(mesh); currentObjects.push(mesh);
        } else {
          const m = new THREE.MeshNormalMaterial();
          const mesh = new THREE.Mesh(planeGeo, m);
          scene.add(mesh); currentObjects.push(mesh);
        }
      } else {
        // unknown geometry type -> show first image fallback
        if(data.visualization_data.images && data.visualization_data.images.length>0){
          const url = data.visualization_data.images[0].url;
          const planeGeo = new THREE.PlaneGeometry(4,3);
          const tex = new THREE.TextureLoader().load(url);
          const m = new THREE.MeshBasicMaterial({map:tex});
          const mesh = new THREE.Mesh(planeGeo,m);
          scene.add(mesh); currentObjects.push(mesh);
        }
      }
    }
  } else {
    // no geometry -> show first image if available
    if(data.visualization_data.images && data.visualization_data.images.length>0){
      const url = data.visualization_data.images[0].url;
      const planeGeo = new THREE.PlaneGeometry(4,3);
      const tex = new THREE.TextureLoader().load(url);
      const m = new THREE.MeshBasicMaterial({map:tex});
      const mesh = new THREE.Mesh(planeGeo,m);
      scene.add(mesh); currentObjects.push(mesh);
    } else {
      document.getElementById('info').innerText += " | No images or geometry returned.";
    }
  }
}

function animate(){
  requestAnimationFrame(animate);
  renderer.render(scene, camera);
}
animate();
</script>
</body>
</html>
"""

@app.get("/", response_class=HTMLResponse)
async def index():
    return HTMLResponse(INDEX_HTML)

@app.post("/api/visualize")
async def api_visualize(req: Request):
    body = await req.json()
    query = body.get("query", "plot x^2")
    context = body.get("context", "")
    res = generate_math_visualization(query, context)
    return JSONResponse(res)

# -----------------------
# Launch server (ngrok) — runs in background thread
# -----------------------
def run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

# Start server thread
thread = Thread(target=run_uvicorn, daemon=True)
thread.start()

# Expose via ngrok so you can open in browser
public_url = ngrok.connect(8000).public_url
print(f"🚀 App is running! Open this in your browser: {public_url}")
print("If you are in Colab, click the link above to open the web UI.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.7/517.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.8/456.8 kB 32.7 MB/s eta 0:00:00


INFO:     Started server process [169]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


ERROR:pyngrok.process.ngrok:t=2025-11-16T15:48:53+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-11-16T15:48:53+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.